<a href="https://colab.research.google.com/github/Whoami666/HSE_Lab/blob/main/clean_models/SMOTE_schiz_models_logreg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd drive
%cd MyDrive
%cd shiz

/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/shiz


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [ ]:
df = pd.read_excel('df_standard_cleaned_21feb.xlsx') 
df1d = pd.read_excel('df_standard_only1dfeatures.xlsx')

df = df.set_index('ID')
df1d = df1d.set_index('ID')
df.head(1)

,sex,tds,chroma_cens_avg,chroma_cens_std,chroma_cqt_avg,chroma_cqt_std,chroma_stft_avg,chroma_stft_std,melspectrogram_avg,melspectrogram_std,...,spectral_flatness_std,spectral_rolloff_avg,spectral_rolloff_std,tempogram_avg,tempogram_std,tonnetz_avg,tonnetz_std,zero_crossing_rate_avg,zero_crossing_rate_std,tds_binary
ID,,,,,,,,,,,,,,,,,,,,,
PD-003,1,0,0.685706,0.428088,0.425039,0.296394,0.339246,0.434412,0.327867,0.348849,...,0.31499,0.397736,0.254215,0.747885,0.719312,0.691479,0.329593,0.197304,0.281692,0


**Models**

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.inspection import permutation_importance

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
def calculate_scores(model, givenx, giveny, xtest):
  kf = KFold(n_splits=10)
  crossval_f1 = cross_val_score(model, givenx, giveny, scoring='f1_macro', cv = kf, verbose=False)
  mean_score = crossval_f1.mean()
  std_score = crossval_f1.std()
  print('f1_macro', mean_score, '±', std_score)

  crossval_accuracy = cross_val_score(model, givenx, giveny, scoring='accuracy', cv = kf, verbose=False)
  mean_score = crossval_accuracy.mean()
  std_score = crossval_accuracy.std()
  print('accuracy', mean_score, '±', std_score)

  crossval_recall = cross_val_score(model, givenx, giveny, scoring='recall_macro', cv = kf, verbose=False)
  mean_score = crossval_recall.mean()
  std_score = crossval_recall.std()
  print('recall', mean_score, '±', std_score)

  if giveny.nunique() == 2:
    y_proba = model.predict_proba(xtest)
    print('roc_auc_score', roc_auc_score(y_test, y_proba[:,1]))
     

In [ ]:
def logreg_class(xtrain, ytrain, xtest, ytest, xgiven, ygiven):
  warnings.filterwarnings('ignore')

  parameters = {
      'penalty' : ['l1','l2'], 
      'C'       : np.logspace(-3,3,7),
      'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
  }

  logreg = LogisticRegression()
  clf = GridSearchCV(logreg,                    # model
                    param_grid = parameters,   # hyperparameters
                    scoring='accuracy',        # metric for scoring
                    cv=10)  
            
  clf.fit(xtrain,ytrain)

  print("Tuned Hyperparameters :", clf.best_params_)
  print("Accuracy :", clf.best_score_, '\n')

  params = clf.best_params_
  logreg = LogisticRegression(solver=params['solver'], max_iter=6000, penalty=params['penalty'], C=params['C']) #, class_weight = 'balanced' 'C': 0.1,  'penalty': 'l2',  'solver': 'liblinear'
  logreg.fit(xtrain, ytrain)
  y_pred = logreg.predict(xtest)

  print(classification_report(ytest,y_pred))
  val_acc = accuracy_score(y_pred, ytest)
  print(val_acc)

  calculate_scores(logreg, xgiven, ygiven, xtest)

**Non-binary**

All features

In [ ]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds # Target variable

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [ ]:
logreg_class(X_train, y_train, X_test, y_test, X, y)

Tuned Hyperparameters : {'C': 1000.0, 'penalty': 'l2', 'solver': 'liblinear'}
Accuracy : 0.9444444444444444 

              precision    recall  f1-score   support

           0       0.96      0.87      0.91        55
           1       0.92      0.97      0.94        60
           2       0.97      1.00      0.98        65

    accuracy                           0.95       180
   macro avg       0.95      0.95      0.95       180
weighted avg       0.95      0.95      0.95       180

0.95
f1_macro 0.7617599248946465 ± 0.27984753088663905
accuracy 0.9266666666666665 ± 0.0837692303332034
recall 0.7846124645144253 ± 0.2588570728629069


Only 1D features

In [ ]:
X = df1d.drop(['tds', 'tds_binary'], axis=1)
y = df1d.tds # Target variable

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [ ]:
logreg_class(X_train, y_train, X_test, y_test, X, y)

Tuned Hyperparameters : {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'}
Accuracy : 0.8481481481481481 

              precision    recall  f1-score   support

           0       0.78      0.69      0.73        55
           1       0.75      0.80      0.77        60
           2       0.97      1.00      0.98        65

    accuracy                           0.84       180
   macro avg       0.83      0.83      0.83       180
weighted avg       0.84      0.84      0.84       180

0.8388888888888889
f1_macro 0.5893836152406249 ± 0.29388781331088154
accuracy 0.7822222222222222 ± 0.15389590838382922
recall 0.5803839044035122 ± 0.31692367945110306


**Binary**

All Features

In [ ]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds_binary # Target variable

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [ ]:
logreg_class(X_train, y_train, X_test, y_test, X, y)

Tuned Hyperparameters : {'C': 10.0, 'penalty': 'l1', 'solver': 'liblinear'}
Accuracy : 0.9444444444444444 

              precision    recall  f1-score   support

           0       0.92      0.85      0.89        55
           1       0.88      0.94      0.91        65

    accuracy                           0.90       120
   macro avg       0.90      0.90      0.90       120
weighted avg       0.90      0.90      0.90       120

0.9
f1_macro 0.6862488313927086 ± 0.23608126083255654
accuracy 0.8966666666666667 ± 0.08621678104251708
recall 0.6897885998521207 ± 0.2624015364974067
roc_auc_score 0.9412587412587413


Only1D


In [ ]:
X = df1d.drop(['tds', 'tds_binary'], axis=1)
y = df1d.tds_binary # Target variable

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [ ]:
logreg_class(X_train, y_train, X_test, y_test, X, y)

Tuned Hyperparameters : {'C': 10.0, 'penalty': 'l1', 'solver': 'liblinear'}
Accuracy : 0.8666666666666666 

              precision    recall  f1-score   support

           0       0.73      0.82      0.77        55
           1       0.83      0.74      0.78        65

    accuracy                           0.78       120
   macro avg       0.78      0.78      0.77       120
weighted avg       0.78      0.78      0.78       120

0.775
f1_macro 0.5151258825412961 ± 0.1557734672426017
accuracy 0.7566666666666667 ± 0.08439325934114777
recall 0.4770134496813262 ± 0.18071549404330278
roc_auc_score 0.8075524475524476
